Primary experiments on the fulfillment of the irrepresentability conditions under shift intervention.

Used to judge if the shift intervention does significantly change the outcome.

### Preparation

In [1]:
import numpy as np
from sympy import *
from sympy.physics.quantum import TensorProduct
import warnings
from scipy.linalg import solve_continuous_lyapunov
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)
init_printing(use_unicode=True)

# ------- Simulation with random zero entries -------
def simulate_M(bernoulli_matrix: Array, seed: int, p: int) -> Array:
    np.random.seed(seed=seed)
    normal_matrix = np.random.normal(0, 1, (p, p))
    temp_M = bernoulli_matrix * normal_matrix
    for i in range(p): # adjust diagonal entries s.t. M stable
        row_sum = np.sum(np.abs(temp_M[i, :])) - np.abs(temp_M[i, i])
        temp_M[i, i] = - np.sum(np.abs(temp_M[i, :]))
    return temp_M

# ------- Simulation -------
def simulate_full_M(seed: int, p: int) -> Array:
    np.random.seed(seed=seed)
    normal_matrix = np.random.normal(0, 1, (p, p))
    for i in range(p): # adjust diagonal entries s.t. M stable
        normal_matrix[i, i] = - np.sum(np.abs(normal_matrix[i, :]))
    return normal_matrix

# ------- commutation matrix -------
# sympy follows column-major style, numpy row-,amjor -> reshape with different order
def comm_mat(m, n):
    w = np.arange(m * n).reshape((m, n), order="F").T.ravel(order="F")
    return np.eye(m * n)[w, :]
def comm_mat_sympy(m, n):
    w = Matrix(range(m * n)).reshape(n, m).T.reshape(m * n, 1)
    return eye(m * n).extract(w, list(range(m * n)))

# ------- matrix A(Sigma) -------
def create_A_Sigma(cov: Array, p: int) -> Array:
    identity = eye(p)
    return np.kron(cov, identity) + np.matmul(np.kron(identity, cov), comm_mat(p, p))
def create_A_Sigma_sympy(cov: Matrix, p: int) -> Matrix:
    identity = eye(p)
    return TensorProduct(cov, identity) + TensorProduct(identity, cov) * comm_mat_sympy(p, p)

# ------- matrix A (intervention) -------
def create_A(A_Sigma: Array, mean: Array, p: int) -> Array:
    identity = mean[0] * eye(p)
    for i in range(1, p):
        identity = np.hstack((identity, mean[i] * eye(p)))
    return np.vstack((A_Sigma, identity))
def create_A_sympy(A_Sigma: Matrix, mean: Matrix, p: int) -> Matrix:
    identity = mean[0] * eye(p)
    for i in range(1, p):
        identity = Matrix.hstack(identity, mean[i] * eye(p))
    return Matrix.vstack(A_Sigma, identity)

# ------- Gram matrix -------
def gram(A: Array) -> Array:
    return np.matmul(np.transpose(A), A)
def gram_sympy(A: Matrix) -> Matrix:
    return A.T * A

# ------- vector g -------
def vec_g(A: Array, vec_C: Array) -> Array:
    return - np.dot(A, vec_C)
def vec_g_sympy(A: Matrix, vec_C: Matrix) -> Matrix:
    return - A * vec_C

# ------- irrepresentability condition -------
def irrep_cond(A: Array, supp: Array, supp_compl: Array) -> float:
    gram_matrix = gram(A=A)
    try:
        matrix_1 = gram_matrix[supp_compl[:, None], supp] # uses broadcasting
        matrix_2 = np.linalg.inv(gram_matrix[np.ix_(supp, supp)])
        return np.linalg.norm(np.matmul(matrix_1, matrix_2), ord=np.inf)
    except np.linalg.LinAlgError:
        return -1

# ------- weak irrepresentability condition -------
def weak_irrep_cond(M: Array, A: Array, supp: Array, supp_compl: Array) -> float:
    gram_matrix = gram(A=A)
    try:
        matrix_1 = gram_matrix[supp_compl[:, None], supp] # uses broadcasting
        matrix_2 = np.linalg.inv(gram_matrix[np.ix_(supp, supp)])
        sign_vec = np.sign(M.flatten(order="F"))[supp]
        return np.linalg.norm(np.matmul(matrix_1, matrix_2) @ sign_vec, ord=np.inf)
    except np.linalg.LinAlgError:
        return -1

num_exp = 100000 # number of true matrices per case
num_sim = 1000 # number of simulations per true matrix

index_intervention = 0
b = 2

### 3 nodes - Irrepresentability condition

In [ ]:
p = 3
Sigma = MatrixSymbol("Sigma", p, p).as_mutable()
cov = Matrix(p, p, lambda i, j: Sigma[min(i,j),max(i,j)])
mean = MatrixSymbol("mu", p, 1).as_mutable()
A_Sigma = create_A_Sigma_sympy(cov=cov, p=p)
A = create_A_sympy(A_Sigma=A_Sigma, mean=mean, p=p)
A

In [ ]:
p = 3
offset = 120899
d = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
full_support = np.arange(p * p)

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_3 = np.empty(shape=(num_exp, len(d)), dtype=object)

for index in range(len(d)):
    for seed in range(offset, 2 + offset):
        np.random.seed(seed=seed)
        bernoulli_matrix = np.random.binomial(1, d[index], (p, p))

        support = (np.nonzero(bernoulli_matrix.flatten() != 0))[0]
        for i in range(p):
            if ((i * p) + i) not in support:
                support = np.append(support, (i * p) + i)
        supp_compl = np.setdiff1d(full_support, support)

        if len(supp_compl) == 0:
            irrep_3[seed - offset, index] = -2
        
        else:
            true_M = simulate_M(bernoulli_matrix=bernoulli_matrix, seed=seed, p=p)

            true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
            true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

            obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
            emp_cov = np.cov(obs, rowvar=False)
            emp_mean = np.mean(obs, axis=0)

            emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
            emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
            emp_A = np.float64(emp_A)

            irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

            irrep_3[seed - offset, index] = irrep_condition

print("Complete graph: ", np.sum(irrep_3 == -2, axis=0))
print("Singular Gram matrix: ", np.sum(irrep_3 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_3 < 1, axis=0) - np.sum(irrep_3 == -2, axis=0) - np.sum(irrep_3 == -1, axis=0)))

### 3 nodes - Weak irrepresentability condition

In [ ]:
p = 3
offset = 120899
d = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
full_support = np.arange(p * p)

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

weak_irrep_3 = np.empty(shape=(num_exp, len(d)), dtype=object)

for index in range(len(d)):
    for seed in range(offset, num_exp + offset):
        np.random.seed(seed=seed)
        bernoulli_matrix = np.random.binomial(1, d[index], (p, p))

        support = (np.nonzero(bernoulli_matrix.flatten() != 0))[0]
        for i in range(p):
            if ((i * p) + i) not in support:
                support = np.append(support, (i * p) + i)
        supp_compl = np.setdiff1d(full_support, support)

        if len(supp_compl) == 0:
            weak_irrep_3[seed - offset, index] = -2
        
        else:
            true_M = simulate_M(bernoulli_matrix=bernoulli_matrix, seed=seed, p=p)

            true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
            true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

            obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
            emp_cov = np.cov(obs, rowvar=False)
            emp_mean = np.mean(obs, axis=0)

            emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
            emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
            emp_A = np.float64(emp_A)

            irrep_condition = weak_irrep_cond(M=true_M, A=emp_A, supp=support, supp_compl=supp_compl)

            weak_irrep_3[seed - offset, index] = irrep_condition

print("Complete graph: ", np.sum(weak_irrep_3 == -2, axis=0))
print("Singular Gram matrix: ", np.sum(weak_irrep_3 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(weak_irrep_3 <= 1, axis=0) - np.sum(weak_irrep_3 == -2, axis=0) - np.sum(weak_irrep_3 == -1, axis=0)))

### 4 nodes - Irrepresentability condition

In [ ]:
p = 4
offset = 120899
d = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
full_support = np.arange(p * p)

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_4 = np.empty(shape=(num_exp, len(d)), dtype=object)

for index in range(len(d)):
    for seed in range(offset, num_exp + offset):
        np.random.seed(seed=seed)
        bernoulli_matrix = np.random.binomial(1, d[index], (p, p))

        support = (np.nonzero(bernoulli_matrix.flatten() != 0))[0]
        for i in range(p):
            if ((i * p) + i) not in support:
                support = np.append(support, (i * p) + i)
        supp_compl = np.setdiff1d(full_support, support)

        if len(supp_compl) == 0:
            irrep_4[seed - offset, index] = -2
        
        else:
            true_M = simulate_M(bernoulli_matrix=bernoulli_matrix, seed=seed, p=p)

            true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
            true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

            obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
            emp_cov = np.cov(obs, rowvar=False)
            emp_mean = np.mean(obs, axis=0)

            emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
            emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
            emp_A = np.float64(emp_A)

            irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

            irrep_4[seed - offset, index] = irrep_condition

print("Complete graph: ", np.sum(irrep_4 == -2, axis=0))
print("Singular Gram matrix: ", np.sum(irrep_4 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_4 < 1, axis=0) - np.sum(irrep_4 == -2, axis=0) - np.sum(irrep_4 == -1, axis=0)))

### 4 nodes - Weak irrepresentability condition

In [ ]:
p = 4
offset = 120899
d = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
full_support = np.arange(p * p)

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

weak_irrep_4 = np.empty(shape=(num_exp, len(d)), dtype=object)

for index in range(len(d)):
    for seed in range(offset, num_exp + offset):
        np.random.seed(seed=seed)
        bernoulli_matrix = np.random.binomial(1, d[index], (p, p))

        support = (np.nonzero(bernoulli_matrix.flatten() != 0))[0]
        for i in range(p):
            if ((i * p) + i) not in support:
                support = np.append(support, (i * p) + i)
        supp_compl = np.setdiff1d(full_support, support)

        if len(supp_compl) == 0:
            weak_irrep_4[seed - offset, index] = -2
        
        else:
            true_M = simulate_M(bernoulli_matrix=bernoulli_matrix, seed=seed, p=p)

            true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
            true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

            obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
            emp_cov = np.cov(obs, rowvar=False)
            emp_mean = np.mean(obs, axis=0)

            emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
            emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
            emp_A = np.float64(emp_A)

            irrep_condition = weak_irrep_cond(M=true_M, A=emp_A, supp=support, supp_compl=supp_compl)

            weak_irrep_4[seed - offset, index] = irrep_condition

print("Complete graph: ", np.sum(weak_irrep_4 == -2, axis=0))
print("Singular Gram matrix: ", np.sum(weak_irrep_4 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(weak_irrep_4 <= 1, axis=0) - np.sum(weak_irrep_4 == -2, axis=0) - np.sum(weak_irrep_4 == -1, axis=0)))

### 5 nodes - Irrepresentability condition

In [ ]:
p = 5
offset = 120899
d = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
full_support = np.arange(p * p)

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_5 = np.empty(shape=(num_exp, len(d)), dtype=object)

for index in range(len(d)):
    for seed in range(offset, num_exp + offset):
        np.random.seed(seed=seed)
        bernoulli_matrix = np.random.binomial(1, d[index], (p, p))

        support = (np.nonzero(bernoulli_matrix.flatten() != 0))[0]
        for i in range(p):
            if ((i * p) + i) not in support:
                support = np.append(support, (i * p) + i)
        supp_compl = np.setdiff1d(full_support, support)

        if len(supp_compl) == 0:
            irrep_5[seed - offset, index] = -2
        
        else:
            true_M = simulate_M(bernoulli_matrix=bernoulli_matrix, seed=seed, p=p)

            true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
            true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

            obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
            emp_cov = np.cov(obs, rowvar=False)
            emp_mean = np.mean(obs, axis=0)

            emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
            emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
            emp_A = np.float64(emp_A)

            irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

            irrep_5[seed - offset, index] = irrep_condition

print("Complete graph: ", np.sum(irrep_5 == -2, axis=0))
print("Singular Gram matrix: ", np.sum(irrep_5 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_5 < 1, axis=0) - np.sum(irrep_5 == -2, axis=0) - np.sum(irrep_5 == -1, axis=0)))

### 5 nodes - Weak irrepresentability condition

In [ ]:
p = 5
offset = 120899
d = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
full_support = np.arange(p * p)

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

weak_irrep_5 = np.empty(shape=(num_exp, len(d)), dtype=object)

for index in range(len(d)):
    for seed in range(offset, num_exp + offset):
        np.random.seed(seed=seed)
        bernoulli_matrix = np.random.binomial(1, d[index], (p, p))

        support = (np.nonzero(bernoulli_matrix.flatten() != 0))[0]
        for i in range(p):
            if ((i * p) + i) not in support:
                support = np.append(support, (i * p) + i)
        supp_compl = np.setdiff1d(full_support, support)

        if len(supp_compl) == 0:
            weak_irrep_5[seed - offset, index] = -2
        
        else:
            true_M = simulate_M(bernoulli_matrix=bernoulli_matrix, seed=seed, p=p)

            true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
            true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

            obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
            emp_cov = np.cov(obs, rowvar=False)
            emp_mean = np.mean(obs, axis=0)

            emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
            emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
            emp_A = np.float64(emp_A)

            irrep_condition = weak_irrep_cond(M=true_M, A=emp_A, supp=support, supp_compl=supp_compl)

            weak_irrep_5[seed - offset, index] = irrep_condition

print("Complete graph: ", np.sum(weak_irrep_5 == -2, axis=0))
print("Singular Gram matrix: ", np.sum(weak_irrep_5 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(weak_irrep_5 <= 1, axis=0) - np.sum(weak_irrep_5 == -2, axis=0) - np.sum(weak_irrep_5 == -1, axis=0)))

## Specify graphs

### 1 edge
Graph with 2 nodes and edge $1\to 2$ present, meaning $m_{21} \neq 0$ and $m_{12} = 0$.

In [2]:
p = 2
offset = 12
index_zero = np.array([[0, 1]])
support = np.array([0, 2, 3])
supp_compl = np.array([1])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_1edge = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_1edge[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_1edge == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_1edge < 1, axis=0) - np.sum(irrep_1edge == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  29864


**2.** Graph with 3 nodes and edges $1\to 2$ and $2\to 3$ present, meaning $m_{21} \neq 0$ and $m_{32} \neq 0$.

In [3]:
p = 3
offset = 1208
index_zero = np.array([[0, 1], [0, 2], [1, 2], [2, 0]])
support = np.array([0, 3, 4, 7, 8])
supp_compl = np.array([1, 2, 5, 6])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_2edges_2 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_2edges_2[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_2edges_2 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_2edges_2 < 1, axis=0) - np.sum(irrep_2edges_2 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  4120


**3.** Graph with 3 nodes and edges $1\to 2$ and $3\to 2$ present, meaning $m_{21} \neq 0$ and $m_{23} \neq 0$.

In [4]:
p = 3
offset = 1208
index_zero = np.array([[0, 1], [0, 2], [2, 0], [2, 1]])
support = np.array([0, 3, 4, 5, 8])
supp_compl = np.array([1, 2, 6, 7])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_2edges_3 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_2edges_3[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_2edges_3 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_2edges_3 < 1, axis=0) - np.sum(irrep_2edges_3 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  20150


### 3 edges - 3 nodes
**1.** Graph with 3 nodes and edges $1\to 2$, $2\to 3$, and $3\to 2$ present, meaning $m_{21} \neq 0$, $m_{32} \neq 0$, and $m_{23} \neq 0$.

Not present are edges $2\to 1$, $1\to 3$, and $3\to 1$, meaning $m_{12} = 0$, $m_{31} = 0$, and $m_{13} = 0$.

In [5]:
p = 3
offset = 1208
index_zero = np.array([[0, 1], [0, 2], [2, 0]])
support = np.array([0, 3, 4, 5, 7, 8])
supp_compl = np.array([1, 2, 6])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_3edges_1 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_3edges_1[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_3edges_1 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_3edges_1 < 1, axis=0) - np.sum(irrep_3edges_1 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  0


**2.** Graph with 3 nodes and edges $2\to 1$, $2\to 3$, and $3\to 2$ present, meaning $m_{12} \neq 0$, $m_{32} \neq 0$, and $m_{23} \neq 0$.

Not present are edges $1\to 2$, $1\to 3$, and $3\to 1$, meaning $m_{21} = 0$, $m_{31} = 0$, and $m_{13} = 0$.

In [6]:
p = 3
offset = 1208
index_zero = np.array([[0, 2], [1, 0], [2, 0]])
support = np.array([0, 1, 4, 5, 7, 8])
supp_compl = np.array([2, 3, 6])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_3edges_2 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_3edges_2[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_3edges_2 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_3edges_2 < 1, axis=0) - np.sum(irrep_3edges_2 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  0


**3.** Graph with 3 nodes and edges $1\to 2$, $2\to 3$, and $3\to 1$ present, meaning $m_{21} \neq 0$, $m_{32} \neq 0$, and $m_{13} \neq 0$.

Not present are edges $1\to 3$, $2\to 1$, and $3\to 2$, meaning $m_{31} = 0$, $m_{12} = 0$, and $m_{23} = 0$.

In [7]:
p = 3
offset = 1208
index_zero = np.array([[0, 1], [1, 2], [2, 0]])
support = np.array([0, 2, 3, 4, 7, 8])
supp_compl = np.array([1, 5, 6])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_3edges_3 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_3edges_3[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_3edges_3 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_3edges_3 < 1, axis=0) - np.sum(irrep_3edges_3 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  53


**4.** Graph with 3 nodes and edges $1\to 2$, $2\to 3$, and $1\to 3$ present, meaning $m_{21} \neq 0$, $m_{31} \neq 0$, and $m_{32} \neq 0$.

Not present are edges $3\to 1$, $2\to 1$, and $3\to 2$, meaning $m_{13} = 0$, $m_{12} = 0$, and $m_{23} = 0$.

In [8]:
p = 3
offset = 1208
index_zero = np.array([[0, 1], [0, 2], [1, 2]])
support = np.array([0, 3, 4, 6, 7, 8])
supp_compl = np.array([1, 2, 5])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_3edges_4 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_3edges_4[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_3edges_4 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_3edges_4 < 1, axis=0) - np.sum(irrep_3edges_4 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  2960


### 3 edges - 4 nodes
**5.** Graph with 4 nodes and edges $1\to 2$, $1\to 4$, and $4\to 3$ present, meaning $m_{21} \neq 0$, $m_{41} \neq 0$, and $m_{34} \neq 0$.

In [9]:
p = 4
offset = 1208
index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 0], [2, 1], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 10, 11, 12, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 8, 9, 13, 14])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_3edges_5 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_3edges_5[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_3edges_5 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_3edges_5 < 1, axis=0) - np.sum(irrep_3edges_5 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  2240


**6.** Graph with 4 nodes and edges $2\to 1$, $1\to 4$, and $4\to 3$ present, meaning $m_{12} \neq 0$, $m_{41} \neq 0$, and $m_{34} \neq 0$.

In [10]:
p = 4
offset = 1208
index_zero = np.array([[0, 2], [0, 3], [1, 0], [1, 2], [1, 3], [2, 0], [2, 1], [3, 1], [3, 2]])
support = np.array([0, 1, 5, 10, 11, 12, 15])
supp_compl = np.array([2, 3, 4, 6, 7, 8, 9, 13, 14])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_3edges_6 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_3edges_6[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_3edges_6 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_3edges_6 < 1, axis=0) - np.sum(irrep_3edges_6 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  2608


**7.** Graph with 4 nodes and edges $2\to 1$, $3\to 1$, and $4\to 1$ present, meaning $m_{12} \neq 0$, $m_{13} \neq 0$, and $m_{14} \neq 0$.

In [11]:
p = 4
offset = 1208
index_zero = np.array([[1, 0], [1, 2], [1, 3], [2, 0], [2, 1], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 2, 3, 5, 10, 15])
supp_compl = np.array([4, 6, 7, 8, 9, 11, 12, 13, 14])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_3edges_7 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_3edges_7[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_3edges_7 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_3edges_7 < 1, axis=0) - np.sum(irrep_3edges_7 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  74043


**8.** Graph with 4 nodes and edges $2\to 1$, $1\to 3$, and $4\to 1$ present, meaning $m_{12} \neq 0$, $m_{31} \neq 0$, and $m_{14} \neq 0$.

In [12]:
p = 4
offset = 1208
index_zero = np.array([[0, 2], [1, 0], [1, 2], [1, 3], [2, 1], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 3, 5, 8, 10, 15])
supp_compl = np.array([2, 4, 6, 7, 9, 11, 12, 13, 14])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_3edges_8 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_3edges_8[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_3edges_8 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_3edges_8 < 1, axis=0) - np.sum(irrep_3edges_8 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  15830


**9.** Graph with 4 nodes and edges $2\to 1$, $2\to 3$, and $4\to 1$ present, meaning $m_{12} \neq 0$, $m_{32} \neq 0$, and $m_{14} \neq 0$.

In [13]:
p = 4
offset = 1208
index_zero = np.array([[0, 2], [1, 0], [1, 2], [1, 3], [2, 0], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 3, 5, 9, 10, 15])
supp_compl = np.array([2, 4, 6, 7, 8, 11, 12, 13, 14])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_3edges_9 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_3edges_9[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_3edges_9 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_3edges_9 < 1, axis=0) - np.sum(irrep_3edges_9 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  32792


**10.** Graph with 4 nodes and edges $1\to 2$, $1\to 3$, and $1\to 4$ present, meaning $m_{21} \neq 0$, $m_{31} \neq 0$, and $m_{41} \neq 0$.

In [14]:
p = 4
offset = 1208
index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 1], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 8, 10, 12, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 9, 11, 13, 14])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_3edges_10 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_3edges_10[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_3edges_10 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_3edges_10 < 1, axis=0) - np.sum(irrep_3edges_10 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  6840


### 4 edges - 4 nodes
**1.** Graph with 4 nodes and edges $1\to 2$, $2\to 3$, $3\to 4$, and $4\to 1$ present, meaning $m_{21} \neq 0$, $m_{32} \neq 0$, $m_{43} \neq 0$, and $m_{14} \neq 0$.

In [15]:
p = 4
offset = 1208
index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 1], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 3, 4, 5, 9, 10, 14, 15])
supp_compl = np.array([1, 2, 6, 7, 8, 11, 12, 13])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_4edges_1 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_4edges_1[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_4edges_1 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_4edges_1 < 1, axis=0) - np.sum(irrep_4edges_1 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  31


**2.** Graph with 4 nodes and edges $1\to 2$, $2\to 1$, $2\to 3$, and $4\to 1$ present, meaning $m_{21} \neq 0$, $m_{12} \neq 0$, $m_{32} \neq 0$, and $m_{14} \neq 0$.

In [16]:
p = 4
offset = 1208
index_zero = np.array([[0, 2], [1, 2], [1, 3], [2, 0], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 3, 4, 5, 9, 10, 15])
supp_compl = np.array([2, 6, 7, 8, 11, 12, 13, 14])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_4edges_2 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_4edges_2[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_4edges_2 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_4edges_2 < 1, axis=0) - np.sum(irrep_4edges_2 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  19175


**3.** Graph with 4 nodes and edges $1\to 2$, $2\to 3$, $3\to 4$, and $1\to 4$ present, meaning $m_{21} \neq 0$, $m_{32} \neq 0$, $m_{43} \neq 0$, and $m_{41} \neq 0$.

In [17]:
p = 4
offset = 1208
index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 0], [2, 3], [3, 1]])
support = np.array([0, 4, 5, 9, 10, 12, 14, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 8, 11, 13])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_4edges_3 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_4edges_3[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_4edges_3 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_4edges_3 < 1, axis=0) - np.sum(irrep_4edges_3 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  343


**4.** Graph with 4 nodes and edges $1\to 2$, $3\to 2$, $3\to 4$, and $1\to 4$ present, meaning $m_{21} \neq 0$, $m_{23} \neq 0$, $m_{43} \neq 0$, and $m_{41} \neq 0$.

In [18]:
p = 4
offset = 1208
index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 3], [2, 0], [2, 1], [2, 3], [3, 1]])
support = np.array([0, 4, 5, 6, 10, 12, 14, 15])
supp_compl = np.array([1, 2, 3, 7, 8, 9, 11, 13])

C = 2 * np.eye(p)
unit_v = np.eye(p)[:, index_intervention]

irrep_4edges_4 = np.empty(shape=(num_exp), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_full_M(seed=seed, p=p)
    true_M[index_zero[:, 0], index_zero[:, 1]] = 0

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)

    irrep_4edges_4[seed - offset] = irrep_condition

print("Singular Gram matrix: ", np.sum(irrep_4edges_4 == -1, axis=0))
print("Irrepresentability condition satisfied: ", (np.sum(irrep_4edges_4 < 1, axis=0) - np.sum(irrep_4edges_4 == -1, axis=0)))

Singular Gram matrix:  0
Irrepresentability condition satisfied:  5762
